In [1]:
import os
import numpy as np

DATASET_DIR = "/media/alperen/Alperen HardDisk/Teknofest 2023/YARISMA 2/applicationdicomanon/"
OUTPUT_DIR = "/media/alperen/Alperen HardDisk/Teknofest 2023/YARISMA 2/PNG VERSIONS/"

# DON'T FORGET TO PUT / TO ENDS OF THE DIRECTORIES

SHAPE = (480, 480)

In [2]:
import cv2
from PIL import Image

def resize(image, new_size, method):
    return cv2.resize(image, new_size, interpolation = method)

In [3]:
import pydicom
from pydicom.pixel_data_handlers.util import apply_modality_lut

def window_dicom(image, center, width):
    windowed_image = (np.clip((image - (center - (width / 2))) / width, 0., 1.0) * 255).astype('uint8')
    return windowed_image

def get_image(dicom):
    image = dicom.pixel_array
    image = apply_modality_lut(image, dicom)
    
    center = dicom.WindowCenter
    
    if(type(center) == pydicom.multival.MultiValue):
        center = center[0]
        
    width = dicom.WindowWidth
    
    if(type(width) == pydicom.multival.MultiValue):
        width = width[0]
            
    image = window_dicom(image, center, width)
            
    if(dicom.PhotometricInterpretation == "MONOCHROME1"):
        image = np.invert(image)
               
    return image

def make_image(file_loc, shape = SHAPE):
    dicom = pydicom.dcmread(file_loc)
    dicom.PixelData = resize(dicom.pixel_array, shape, cv2.INTER_CUBIC).astype('uint16').tobytes()
    dicom.Rows, dicom.Columns = shape
    
    dicom.file_meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian
    
    return get_image(dicom)

In [4]:
import matplotlib.pyplot as plt

plt.style.use('ggplot')

def show_image(image, show_now = True):
    if(show_now):
        plt.figure(figsize = (10, 10))
        
    plt.imshow(image, cmap = plt.cm.gray, vmin = 0, vmax = 255)
        
    plt.xticks(())
    plt.yticks(())
    
    if(show_now):     
        plt.show()
        plt.close()
        
def show_all(images):
    row_cnt = len(images)
    col_cnt = len(images[0])
    
    plt.figure(figsize = (col_cnt * 10, row_cnt * 10))
    
    for row in range(row_cnt):
        for col in range(col_cnt):
            plt.subplot(row_cnt, col_cnt, row * col_cnt + (col + 1))
            show_image(images[row][col], False)
            
    plt.show()
    plt.close()

In [6]:
folders = os.listdir(DATASET_DIR)

folders = sorted(folders)

for i, patient in enumerate(folders, 0):    
    if(i % 10 == 0):
        print(i)
        
    if(not os.path.exists(OUTPUT_DIR + patient)):
        os.makedirs(OUTPUT_DIR + patient)
        
    for file in os.listdir(DATASET_DIR + patient):
        fixed = make_image(DATASET_DIR + patient + "/" + file)
        
        im = Image.fromarray(fixed)
        
        im.save(OUTPUT_DIR + patient + "/" + file[:-4] + ".png")

0

10

20

30

40

50

60

70

80

90

100

110

120

130

140

150

160

170

180

190

200

210

220

230

240

250

260

270

280

290

300

310

320

330

340

350

360

370

380

390

400

410

420

430

440

450

460

470

480

490
